# Load Data

In [2]:
import numpy as np
import pandas as pd
import glob
import json

import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [5]:
# meta_df = pd.read_csv("metadata.csv")

In [8]:
# if you were to do this individually
# meta_df['pubmed_id'] = meta_df['pubmed_id'].astype(str)


In [12]:
meta_df = pd.read_csv('metadata.csv', dtype ={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str,
    'doi' : str
})

## Fetch All of JSON File Path

In [23]:
ls

COVID.DATA.LIC.AGMT.pdf  comm_use_subset/         metadata.readme
README.md                custom_license/          noncomm_use_subset/
Untitled.ipynb           json_schema.txt
biorxiv_medrxiv/         metadata.csv


In [27]:
pwd

'/Users/johnrick/Desktop/DataScienceLearn/blog_covid19'

In [34]:
all_json = glob.glob(f'{"/Users/johnrick/Desktop/DataScienceLearn/blog_covid19"}/**/*.json', recursive=True)

In [35]:
len(all_json)

33375

# Create Helper Functions

In [37]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            #abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            #body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'
first_row=FileReader(all_json[0])
print(first_row)

86a998617c077f4fe2ab26214995a3548fbc0fc5: The recent emergence of the Middle East respiratory syndrome (MERS)-CoV, a close relative of the Severe Acute respiratory syndrome (SARS)-CoV, both of which caused a lethal respiratory infection in hu... While most CoVs cause the common cold in humans, infection with two recently emerged CoVs, SARS-CoV and MERS-CoV, resulted in more severe pulmonary disease with alarmingly high case fatality rates [1]...


In [38]:
def get_breaks(content, length):
    data = ""
    words = content.split(' ')
    total_chars = 0
    
    #add break every length characters
    for i in range(len(words)):
        total_chars += len(words[i])
        if total_chars > length:
            data = data + "<br>" + words[i]
            total_chars = 0
        else:
            data = data + " " + words[i]
    return data